<a href="https://colab.research.google.com/github/wndgua0468/machine_learning/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_14%EC%A3%BC%EC%B0%A8_3%EB%B2%88%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative to its
# difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# WARNING: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure. You do not need them to solve the question.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# COMPUTER VISION WITH CNNs
#
# Create and train a classifier to classify images between three categories
# of beans using the beans dataset.
# ==============================================================================
# ABOUT THE DATASET
#
# Beans dataset has images belonging to 3 classes as follows:
# 2 disease classes (Angular leaf spot, bean rust)
# 1 healthy class (healthy).
# The images are of different sizes and have 3 channels.
# ==============================================================================
#
# INSTRUCTIONS
#
# We have already divided the data for training and validation.
#
# Complete the code in following functions:
# 1. preprocess()
# 2. solution_model()
#
# Your code will fail to be graded if the following criteria are not met:
# 1. The input shape of your model must be (300,300,3), because the testing
#    infrastructure expects inputs according to this specification. You must
#    resize all the images in the dataset to this size while pre-processing
#    the dataset.
# 2. The last layer of your model must be a Dense layer with 3 neurons
#    activated by softmax since this dataset has 3 classes.
#
# HINT: Your neural network must have a validation accuracy of approximately
# 0.75 or above on the normalized validation dataset for top marks.
#


import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import ModelCheckpoint

# Use this constant wherever necessary
IMG_SIZE = 300

# This function normalizes and resizes the images.

# COMPLETE THE CODE IN THIS FUNCTION
def preprocess(image, label):
    # RESIZE YOUR IMAGES HERE (HINT: After resizing the shape of the images
    # should be (300, 300, 3).
    # NORMALIZE YOUR IMAGES HERE (HINT: Rescale by 1/.255))
    image1= tf.image.resize(image, size=(300,300) )
    image2 = image1/255
    return image2, label


# This function loads the data, normalizes and resizes the images, splits it into
# train and validation sets, defines the model, compiles it and finally
# trains the model. The trained model is returned from this function.

# COMPLETE THE CODE IN THIS FUNCTION.
def solution_model():
    # Loads and splits the data into training and validation splits using tfds.
    (ds_train, ds_validation), ds_info = tfds.load(
        name='beans',
        split=['train', 'validation'],
        as_supervised=True,
        with_info=True)

    BATCH_SIZE = 32 # YOUR CODE HERE

    # Resizes and normalizes train and validation datasets using the
    # preprocess() function.
    # Also makes other calls, as evident from the code, to prepare them for
    # training.
    ds_train = ds_train.map(preprocess).cache().shuffle(
        ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(
        tf.data.experimental.AUTOTUNE)
    ds_validation = ds_validation.map(preprocess).batch(BATCH_SIZE).cache(

    ).prefetch(tf.data.experimental.AUTOTUNE)

    # Code to define the model
    model = tf.keras.models.Sequential([

        # If you don't adhere to the instructions in the following comments,
        # tests will fail to grade your model:
        # The input layer of your model must have an input shape of
        # (300,300,3).
        # Make sure that your last layer has 3 (number of classes) neurons
        # activated by softmax.
        Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        # 2D -> 1D로 변환을 위하여 Flatten 합니다.
        Flatten(),
        # 과적합 방지를 위하여 Dropout을 적용합니다. 학습시에 큰 노드 50%만 사용
        Dropout(0.5),
        Dense(512, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax'),
    ])

    # Code to compile and train the model
    model.compile(
        # YOUR CODE HERE
        optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc']
    )
    epochs = 10
    model.fit(
        # YOUR CODE HERE
     ds_train,
                    validation_data=(ds_validation),
                    epochs=epochs
    )
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel_3.h5")

In [ ]:
import urllib
import zipfile

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip'
urllib.request.urlretrieve(url, 'satellitehurricaneimages.zip')

with zipfile.ZipFile('satellitehurricaneimages.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rescale=1/255.,
                               horizontal_flip=True,
                               rotation_range=35,
                               zoom_range=0.2
                               )
train_dir = '/content/train'
train_gen = image_gen.flow_from_directory(train_dir,
                                        target_size=(300, 300),
                                        batch_size=128,
                                        class_mode='binary'
                                        )

In [ ]:
class_labels = ['damage', 'no_damage']
batch=next(train_gen)
images, labels = batch[0], batch[1]

plt.figure(figsize=(16,8))
for i in range(32):

    ax = plt.subplot(4,8,i+1)
    plt.imshow(images[i])
    plt.title(class_labels[labels[i].astype(np.int)])
    plt.axis('off')
plt.tight_layout()
plt.show()